# Using Tools
This notebook demonstrates how we can use tools to gather information and provide more accurate responses. We'll build a weather-checking mock as an example.

## What we'll learn:
- Basic interaction with Language Models (LLM)
- How to create and use tools with AI
- The complete flow of an AI agent using tools
- Understanding the message flow in a tool-enabled conversation

### Setup

In [13]:
import json
import os
from dotenv import load_dotenv
from lib.messages import UserMessage, SystemMessage, ToolMessage  # Different message types
from lib.tooling import tool  # Tool decorator for creating AI tools
from lib.llm import LLM  # Our Language Model wrapper

In [14]:
load_dotenv()

True

In [15]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
BASE_URL = "https://openai.vocareum.com/v1"

chat_model = LLM(api_key=OPENAI_API_KEY, base_url=BASE_URL)

## Basic LLM Interaction
Before we dive into tools, let's understand how to interact with our Language Model in its simplest form. 
There are two main ways to communicate with the model:

In [16]:
# Method 1: Simple single-turn query
response = chat_model.invoke("What is an AI Agent?")
print("Single Query Response:\n", response.content)

Single Query Response:
 An AI agent is a system or program that uses artificial intelligence techniques to perform tasks autonomously or semi-autonomously. These agents can perceive their environment, make decisions, and take actions based on their programming and the data they receive. AI agents can be categorized into various types based on their capabilities and functionalities:

1. **Reactive Agents**: These agents respond to specific stimuli in their environment without maintaining a memory of past experiences. They operate based on predefined rules or algorithms.

2. **Deliberative Agents**: These agents have a model of the world and can plan and reason about their actions. They can consider future states and make decisions based on goals and objectives.

3. **Learning Agents**: These agents can improve their performance over time by learning from experiences. They often use machine learning techniques to adapt to new information and optimize their actions.

4. **Autonomous Agent

In [17]:
# Method 2: Multi-turn conversation with specific roles
messages = [
    SystemMessage(content="You're an OpenAI API specialist"),
    UserMessage(content="What is Function Calling?")
]
response = chat_model.invoke(messages)
print("\nStructured Conversation Response:\n", response.content)


Structured Conversation Response:
 Function calling is a programming concept where a function is executed or invoked in order to perform a specific task or computation. In programming, a function is a reusable block of code that takes input (arguments), processes it, and often returns an output (result). 

### Key Aspects of Function Calling:

1. **Definition**: Before a function can be called, it must be defined. This includes specifying the function's name, parameters (if any), and the code that will be executed when the function is called.

2. **Parameters and Arguments**: Functions can take parameters, which are variables that act as placeholders for the values you pass to the function when you call it. The actual values passed are called arguments.

3. **Return Value**: Functions can return a value using a return statement. This allows the result of the function to be used elsewhere in the program.

4. **Syntax**: The syntax for calling a function varies by programming language b

## Building an AI Tool
Now let's make our AI more capable by giving it a tool to check the weather. 
This demonstrates how we can extend AI capabilities beyond just conversation.

### Understanding the Tool Structure:
1. We use the `@tool` decorator to mark a function as an AI tool
2. The tool needs clear documentation and typed parameters
3. The tool should return structured data

In [18]:
@tool
def get_weather(city: str):
    """Get the current temperature for a city.
    
    Args:
        city (str): Name of the city to check weather for
        
    Returns:
        dict: Contains temperature information for the requested city
    """
    # In a real application, this would call a weather API
    mock_weather = {
        "São Paulo": "28°C",
        "Oslo": "-3°C",
        "New York": "15°C",
        "Tokyo": "22°C"
    }
    return {"temperature": mock_weather.get(city, "Unknown")}

In [19]:
# Bind the tool to an LLM
chat_model_with_tools = LLM(tools=[get_weather], api_key=OPENAI_API_KEY, base_url=BASE_URL)

## Understanding the Tool Usage Flow
Let's break down how the AI uses tools step by step:

1. User asks a question about weather
2. AI recognizes the need to use the weather tool
3. AI makes a tool call
4. Tool executes and returns results
5. AI processes the tool's response
6. AI formulates a natural language response

Let's see this in action:

In [20]:
# Set up our system with clear instructions
messages = [
    SystemMessage(
        content="You are a helpful assistant that can access a tool to get current temperature " 
                "for cities. Use the tool whenever someone asks about the weather or temperature " 
                "in a specific location. Infor the user if you don't know the answer."
    ),
    UserMessage(content="How cold is it in Oslo?")
]

In [21]:
# AI decides to use the weather tool
ai_message = chat_model_with_tools.invoke(messages)
ai_message

AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_DFpuhyjPdz9KaLSmK6rcW3xv', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')])

In [22]:
# Check messages structure
messages.append(ai_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_DFpuhyjPdz9KaLSmK6rcW3xv', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')])]

In [23]:
# Tool call id will be required later when creating the ToolMessage
tool_call_id = messages[-1].tool_calls[0].id
tool_call_id

'call_DFpuhyjPdz9KaLSmK6rcW3xv'

In [24]:
# Extract the arguments
args = json.loads(messages[-1].tool_calls[0].function.arguments)
args

{'city': 'Oslo'}

In [25]:
# Execute the tool with the AI's requested parameters
tool_result = get_weather(**args)
tool_result

{'temperature': '-3°C'}

In [26]:
# Create a tool response message
tool_message = ToolMessage(
    content=tool_result["temperature"], 
    tool_call_id=tool_call_id, 
    name="get_weather"
)
tool_message

ToolMessage(content='-3°C', role='tool', tool_call_id='call_DFpuhyjPdz9KaLSmK6rcW3xv', name='get_weather')

In [27]:
# Check messages structure
messages.append(tool_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_DFpuhyjPdz9KaLSmK6rcW3xv', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')]),
 ToolMessage(content='-3°C', role='tool', tool_call_id='call_DFpuhyjPdz9KaLSmK6rcW3xv', name='get_weather')]

In [28]:
# Let AI formulate final response
ai_message = chat_model_with_tools.invoke(messages)
ai_message

AIMessage(content='The current temperature in Oslo is -3°C.', role='assistant', tool_calls=None)

In [29]:
# Check messages structure
messages.append(ai_message)
messages

[SystemMessage(content="You are a helpful assistant that can access a tool to get current temperature for cities. Use the tool whenever someone asks about the weather or temperature in a specific location. Infor the user if you don't know the answer.", role='system'),
 UserMessage(content='How cold is it in Oslo?', role='user'),
 AIMessage(content=None, role='assistant', tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_DFpuhyjPdz9KaLSmK6rcW3xv', function=Function(arguments='{"city":"Oslo"}', name='get_weather'), type='function')]),
 ToolMessage(content='-3°C', role='tool', tool_call_id='call_DFpuhyjPdz9KaLSmK6rcW3xv', name='get_weather'),
 AIMessage(content='The current temperature in Oslo is -3°C.', role='assistant', tool_calls=None)]